In [1]:
import tensorflow as tf
import keras 
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import sys
import random
import os                      
import numpy as np              
import pandas as pd             
import torch             
from PIL import Image, ImageOps, ImageFilter
import matplotlib.pyplot as plt 
import torch.nn as nn           
from torch.utils.data import DataLoader  
from PIL import Image          
import torch.nn.functional as F 
import torchvision.transforms as transforms   
from torchvision.utils import make_grid    
from torchvision.datasets import ImageFolder  
from torch.utils.data import random_split
from tensorflow.keras.layers import Input

%matplotlib inline

2024-03-17 10:20:03.923863: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 10:20:03.923964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 10:20:04.063517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout

In [3]:
input_shape = (960, 960, 3)
input_layer = Input(shape=input_shape)
model = keras.applications.EfficientNetV2S(input_tensor=input_layer,weights='imagenet', include_top=True)

87549136/87549136 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [4]:
for layer in model.layers[0:5]:
    layer.trainable = False
    
    
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 rescaling False
2 stem_conv False
3 stem_bn False
4 stem_activation False
5 block1a_project_conv True
6 block1a_project_bn True
7 block1a_project_activation True
8 block1a_add True
9 block1b_project_conv True
10 block1b_project_bn True
11 block1b_project_activation True
12 block1b_drop True
13 block1b_add True
14 block2a_expand_conv True
15 block2a_expand_bn True
16 block2a_expand_activation True
17 block2a_project_conv True
18 block2a_project_bn True
19 block2b_expand_conv True
20 block2b_expand_bn True
21 block2b_expand_activation True
22 block2b_project_conv True
23 block2b_project_bn True
24 block2b_drop True
25 block2b_add True
26 block2c_expand_conv True
27 block2c_expand_bn True
28 block2c_expand_activation True
29 block2c_project_conv True
30 block2c_project_bn True
31 block2c_drop True
32 block2c_add True
33 block2d_expand_conv True
34 block2d_expand_bn True
35 block2d_expand_activation True
36 block2d_project_conv True
37 block2d_project_bn True
38 block

In [5]:
model = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)
model.input

[<KerasTensor shape=(None, 960, 960, 3), dtype=float32, sparse=None, name=keras_tensor>]

In [6]:
x = model.output
x = Dense(3, activation='softmax')(x)
model = Model(inputs=model.input, outputs=x)

In [7]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('EFNetV2S-960-ft.keras', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [8]:
import glob
data = glob.glob('/kaggle/input/transcend/train_images/train_images/*.*')

In [9]:
data[:5]

['/kaggle/input/transcend/train_images/train_images/IMG_160.jpg',
 '/kaggle/input/transcend/train_images/train_images/IMG_21.jpg',
 '/kaggle/input/transcend/train_images/train_images/IMG_163.jpg',
 '/kaggle/input/transcend/train_images/train_images/IMG_151.jpg',
 '/kaggle/input/transcend/train_images/train_images/IMG_190.jpg']

In [10]:
ann = pd.read_csv('/kaggle/input/transcend/train_annotations.csv')
ann

,Name,State
0,IMG_1.jpg,malignant
1,IMG_2.jpg,non-nodule
2,IMG_3.jpg,non-nodule
3,IMG_4.jpg,malignant
4,IMG_5.jpg,non-nodule
...,...,...
195,IMG_196.jpg,malignant
196,IMG_197.jpg,malignant
197,IMG_198.jpg,non-nodule
198,IMG_199.jpg,malignant


In [11]:
from PIL import Image, ImageOps, ImageFilter
image = Image.open('/kaggle/input/transcend/train_images/train_images/IMG_121.jpg').convert('RGB')

In [12]:
image.size

(2048, 2048)

In [13]:
ann['State'].replace({'benign': 0, 'non-nodule': 1, 'malignant': 2}, inplace=True)
ann

/tmp/ipykernel_34/1362740101.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ann['State'].replace({'benign': 0, 'non-nodule': 1, 'malignant': 2}, inplace=True)
/tmp/ipykernel_34/1362740101.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ann['State'].replace({'benign': 0, 'non-nodule': 1, 'ma

,Name,State
0,IMG_1.jpg,2
1,IMG_2.jpg,1
2,IMG_3.jpg,1
3,IMG_4.jpg,2
4,IMG_5.jpg,1
...,...,...
195,IMG_196.jpg,2
196,IMG_197.jpg,2
197,IMG_198.jpg,1
198,IMG_199.jpg,2


In [14]:
labels = []

for image_name in data:
    img_file_name = image_name.split('/')[-1]
    label = ann[ann['Name'] == img_file_name]['State'].values[0]
    labels.append(label)

In [15]:
from PIL import Image, ImageOps, ImageFilter
import numpy as np
import os
from sklearn.utils import shuffle

images = []

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    width, height = image.size
    pad = 960
    image = image.resize((pad, pad))
    image = image.filter(ImageFilter.SHARPEN)
    image = np.array(image)
    images.append(image)


for i in data:
    preprocess_image(i)

In [16]:
images = np.array(images)
images.shape

(200, 960, 960, 3)

In [17]:
labels= np.array(labels)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [19]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.Adam(), metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size = 4, epochs=30, validation_data=(X_test,y_test), callbacks=[lr_reduce,checkpoint])

Epoch 1/30


I0000 00:00:1710671102.903035     104 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710671103.107165     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.4504 - loss: 1.1437

W0000 00:00:1710671129.129440     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_accuracy improved from -inf to 0.42500, saving model to EFNetV2S-960-ft.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 272s 732ms/step - accuracy: 0.4497 - loss: 1.1435 - val_accuracy: 0.4250 - val_loss: 1.0620 - learning_rate: 0.0010
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.5608 - loss: 1.0332
Epoch 2: val_accuracy improved from 0.42500 to 0.45000, saving model to EFNetV2S-960-ft.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 454ms/step - accuracy: 0.5587 - loss: 1.0342 - val_accuracy: 0.4500 - val_loss: 1.0002 - learning_rate: 0.0010
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.5100 - loss: 1.0440
Epoch 3: val_accuracy did not improve from 0.45000
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 401ms/step - accuracy: 0.5100 - loss: 1.0440 - val_accuracy: 0.4000 - val_loss: 1.1209 - learning_rate: 0.0010
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.3751 - loss: 1.1145
Epoch 4: val_accuracy did not improve from 0.45000
40/40 ━━━━━━━━━━━━━━━━━━━━ 

In [21]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/EFNetV2S-960-ft.keras')

In [22]:
test = glob.glob('/kaggle/input/transcend/test_images-20240316T153237Z-001/test_images/*.*')

In [23]:
test_images = []
names = []

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    img_file_name = image_path.split('/')[-1]
    width, height = image.size
    pad = 960
    image = image.resize((pad, pad))
    image = image.filter(ImageFilter.SHARPEN)
    image = np.array(image)
    test_images.append(image)
    names.append(img_file_name)


for i in test:
    preprocess_image(i)
    
test_images = np.array(test_images)

In [26]:
y_pred = model.predict(test_images)

2/2 ━━━━━━━━━━━━━━━━━━━━ 70s 22s/step


In [27]:
y_pred[:5]

array([[9.1911666e-03, 9.8781234e-01, 2.9965779e-03],
       [1.4736951e-02, 9.0004150e-03, 9.7626263e-01],
       [7.8480490e-02, 3.3637309e-03, 9.1815579e-01],
       [2.4744393e-05, 7.7888179e-01, 2.2109357e-01],
       [2.8585145e-02, 9.6454728e-01, 6.8676290e-03]], dtype=float32)

In [28]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 2, 2, 1, 1, 2, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 2, 2,
       1, 0, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 0, 1])

In [29]:
len(y_pred)

40

In [30]:
sub= pd.DataFrame()
sub['Name'] = names
sub['State'] = y_pred
sub['State'].replace({0: 'benign', 1 : 'non-nodule', 2: 'malignant'}, inplace=True)
sub

/tmp/ipykernel_34/2320143535.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sub['State'].replace({0: 'benign', 1 : 'non-nodule', 2: 'malignant'}, inplace=True)


,Name,State
0,IMG_233.jpg,non-nodule
1,IMG_212.jpg,malignant
2,IMG_236.jpg,malignant
3,IMG_228.jpg,non-nodule
4,IMG_237.jpg,non-nodule
5,IMG_204.jpg,malignant
6,IMG_224.jpg,benign
7,IMG_234.jpg,benign
8,IMG_220.jpg,malignant
9,IMG_207.jpg,malignant


In [31]:
sub = sub.sort_values(by='Name')
sub

,Name,State
11,IMG_201.jpg,benign
39,IMG_202.jpg,non-nodule
19,IMG_203.jpg,non-nodule
5,IMG_204.jpg,malignant
26,IMG_205.jpg,non-nodule
30,IMG_206.jpg,benign
9,IMG_207.jpg,malignant
23,IMG_208.jpg,benign
33,IMG_209.jpg,non-nodule
37,IMG_210.jpg,benign


In [32]:
sub.to_csv("task1.csv")